In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import os
import pylcs
from typing import Union, List, Optional
from pprint import pprint

import transformers
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                         pipeline, TrainingArguments, Trainer,
                         DataCollatorForTokenClassification, EarlyStoppingCallback)

import datasets
from datasets import Dataset, Features, Sequence, Value, ClassLabel, load_dataset, load_from_disk

import torch
import evaluate
import re


print(transformers.__version__)

import os
os.environ['WANDB_DISABLED'] = 'true'

task = "ner" # Should be one of "ner", "pos" or "chunk"
# model_checkpoint = "distilbert-base-uncased"
batch_size = 16
seqeval = evaluate.load("seqeval")

4.51.3


In [15]:
path_to_model = "/home/mikhail/Documents/pandan_study/vkr/vulns_scanner/mikhail_code/models/nuner_as_tok_clf_190425/best_model"
final_tokenizer = AutoTokenizer.from_pretrained(path_to_model, use_fast=True, add_prefix_space=True, local_files_only=True)
final_model = AutoModelForTokenClassification.from_pretrained(path_to_model, local_files_only=True)


In [ ]:
models/nuner_as_tok_clf_190425/best_model

In [16]:
s = "Auth. (contributor+) Stored Cross-Site Scripting (XSS) vulnerability in Julien Berthelot / MPEmbed WP Matterport Shortcode plugin <= 2.1.4 versions."
token_classifier = pipeline(
    "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
)
res = token_classifier(s)
for i, r in enumerate(res):
    # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
    print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '   Prob: ' + str(r['score']))

Device set to use cpu


Entity: vendor   Word:  Julien   Prob: 0.9925166
Entity: product   Word:  WP Matterport Shortcode   Prob: 0.9509382
Entity: version   Word:  <= 2.1.4   Prob: 0.99982387


/home/mikhail/.local/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [10]:
def run(input_string: str) -> str:
    token_classifier = pipeline(
        "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
    )
    res = token_classifier(input_string)
    output = ''
    for i, r in enumerate(res):
        # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
        output += str(f'{i+1}. ' + 'Entity: ' + r['entity_group'] + '   Word: ' + r['word'] + '   Prob: ' + str(r['score']) + '\n')
    # print(output)
    if output == '':
        output = 'No NER found'
    return output

In [11]:
s = "The React Developer Tools extension registers a message listener with window.addEventListener('message', <listener>) in a content script that is accessible to any webpage that is active in the browser. Within the listener is code that requests a URL derived from the received message via fetch(). The URL is not validated or sanitised before it is fetched, thus allowing a malicious web page to arbitrarily fetch URL’s via the victim's browser."

In [12]:
run(s)

Device set to use cpu


'1. Entity: version   Word:  before it   Prob: 0.784055\n'